# TMDB (The Movie DB)

## Collect Data From TMDB By IMDB List

In [0]:
!pip install tmdbsimple

In [0]:
import tmdbsimple as tmdb
tmdb.API_KEY = '9bce8bd4cb035c7799b98338d9e227c0'

fileName ='allData.txt'
train_movie_id_list = [line.rstrip('\n') for line in open(fileName)]
imdb_list = train_movie_id_list

len(imdb_list)
general_list = []

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import urllib.request, json

big_counter = -1

for imdb_id in imdb_list:
  big_counter = big_counter + 1
  if big_counter >= 133434 and  big_counter <= 226950:
    try:
        with urllib.request.urlopen("https://api.themoviedb.org/3/find/"+imdb_id+"?api_key=9bce8bd4cb035c7799b98338d9e227c0&language=en-US&external_source=imdb_id") as url:
            data = json.loads(url.read().decode())
            if len(data["movie_results"]) != 0:
                moviedb_id = data["movie_results"][0]['id']
                
                movie = tmdb.Movies(moviedb_id)

                response = movie.info()
                movie_credits_dic = movie.credits(append_to_response='')

                movie_list = []

                movie_list.append("Movie_Name: " + movie.title)
                movie_list.append("Movie_Id: " + movie.imdb_id)

                counter = 0
                for one_movie in movie_credits_dic["cast"]:
                    if "character" in one_movie:
                        movie_list.append("Cast: " + one_movie["name"])
                        counter  = counter + 1

                counter = 0
                for one_movie in movie_credits_dic["crew"]:
                    if "job" in one_movie:
                        if one_movie["job"]=="Director":
                            movie_list.append("Director: " + one_movie["name"])
                    counter  = counter + 1

                general_list.append(movie_list)
        

    except Exception:
        print("Problem Cikti!!!")
        continue

In [0]:
with open('cast_director.txt', 'w') as f:
    for item in general_list:
        f.write("%s\n" % item)

In [0]:
!cp "/content/cast_director.txt" "/content/drive/My Drive/"

## Combine Splitted Datasets Above

In [0]:
!cp "/content/drive/My Drive/cast_director.txt" "/content"
!cp "/content/drive/My Drive/cast_director2.txt" "/content"

first = [line.rstrip('\n') for line in open("cast_director.txt")]
second = [line.rstrip('\n') for line in open("cast_director2.txt")]

new_list = first + second

len(new_list)

In [0]:
new_list = [line.rstrip('\n') for line in open("cast_director.txt")]
len(new_list)

104241

In [0]:
import ast

gen_list = []

for element in new_list:
  gen_list.append(ast.literal_eval(element))

In [0]:
cast_director = []

for element in gen_list:
  control_bool_1 = False
  control_bool_2 = False
  for inner_element in element:
    if "Cast:" in inner_element:
      control_bool_1 = True
    if "Director:" in inner_element:
      control_bool_2 = True

  if (control_bool_1==True) and (control_bool_2==True):
    for inner_element in element:
      if "Director:" in inner_element:
        for inner_element_2 in element:
          if "Cast:" in inner_element_2:
            cast_director.append(inner_element.replace('Director: ','') + "---" + inner_element_2.replace('Cast: ',''))

In [0]:
with open('cast_director_edges.txt', 'w') as f:
    for item in cast_director:
        f.write("%s\n" % item)

## Count Edges & Edit Edges

In [0]:
edge_list = [line.rstrip('\n') for line in open("cast_director_edges.txt")]

len(edge_list)

937579

In [0]:
import pandas as pd

edgesData = pd.read_csv('cast_director_edges.txt', sep='---', header=None)
edgesData.columns = ["director", "movie_star"]
edgesData

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,director,movie_star
0,James Mangold,Meg Ryan
1,James Mangold,Hugh Jackman
2,James Mangold,Liev Schreiber
3,James Mangold,Breckin Meyer
4,James Mangold,Natasha Lyonne
...,...,...
937574,Jixing Wang,Li Xuejian
937575,Jixing Wang,Jiang Wenli
937576,Jixing Wang,Li Youbin
937577,Jixing Wang,Guo Jinglin


In [0]:
counted = edgesData.groupby(edgesData.columns.tolist()).size().reset_index().\
    rename(columns={0:'weight'})

In [0]:
counted

,director,movie_star,weight
0,'Evil' Ted Smith,Ardy Brent Carlson,1
1,'Evil' Ted Smith,Chris Sheets,1
2,'Evil' Ted Smith,Glen Levy,1
3,'Evil' Ted Smith,Heather Ashley Chase,1
4,'Evil' Ted Smith,Isis McKenzie,1
...,...,...,...
875889,최신규,Shin Yong-woo,1
875890,최신규,Um Sang-hyun,1
875891,최신규,Yang Jeong-hwa,1
875892,최신규,김용준,1


In [0]:
counted = counted.sort_values(by=['weight'])

In [0]:
counted.to_csv("counted_TMDB.txt")